In [948]:
import numpy as np
import cv2
from collections import Counter
import pandas as pd
from skimage import measure,draw,data

In [964]:
class CutImg:
    def __init__(self,Img_url,Left_right,ThetaSmall,ThetaBig,PxNum,LenNum,GapNum):
        self.img_url="D:/Data/DigImagDetct/{}.bmp".format(Img_url)
        self.canny_url="BigImageResult/cannyImg{}.bmp".format(Img_url)
        self.line_url="BigImageResult/lineImg{}.bmp".format(Img_url)
        self.cut_url="BigImageResult/cutImg{}.bmp".format(Img_url)
        self.left_right=Left_right
        self.thetaSmall=ThetaSmall
      self.thetaBig=ThetaBig
        self.pxNum=PxNum
        self.lenNum=LenNum
        self.gapNum=GapNum
        
    def readImg(self):
        img = cv2.imread(self.img_url, 0)
        return img
    
    def setImg(self):
        img=self.readImg()
        canny=cv2.Canny(img,self.thetaSmall,self.thetaBig)
        cv2.imwrite(self.canny_url, canny)
        lines=cv2.HoughLines(canny, 1,np.pi/180,self.pxNum,self.lenNum,self.gapNum)
        if lines is not None:
            for l in lines:
                rho,theta=l[0]
                x=np.cos(theta)
                y=np.sin(theta)
                if x==1:
                    x0=rho/x
                    y0=y*rho
                    x1=int(x0+2250*(-y))
                    y1 = int(y0 + 2250 * (x))
                    x2 = int(x0 - 2250 * (-y))
                    y2 = int(y0 - 2250 * (x))
                    cv2.line(img, (x1, y1), (x2, y2), (0 ,255,0 ), 1)
                    cv2.imwrite(self.line_url, img)
                    break
            if self.left_right=="l":
                cut_img=img[0:img.shape[0],0:x1]
            elif self.left_right=="r":
                cut_img=img[0:img.shape[0],x1:img.shape[1]]
            cv2.imwrite(self.cut_url, cut_img)
        return 
    
    def shapeImg(self):
        img=self.readImg()
        canny=cv2.Canny(img,120,300)
        cv2.imwrite(self.canny_url, canny)
        lines=cv2.HoughLines(canny, 1,np.pi/180,50,100,10)
        if lines is not None:
            for l in lines:
                rho,theta=l[0]
                x=np.cos(theta)
                y=np.sin(theta)
                if x==1:
                    x0=rho/x
                    y0=y*rho
                    x1=int(x0+2250*(-y))
                    y1 = int(y0 + 2250 * (x))
                    x2 = int(x0 - 2250 * (-y))
                    y2 = int(y0 - 2250 * (x))
                    cv2.line(img, (x1, y1), (x2, y2), (0 ,255,0 ), 1)
                    cv2.imwrite(self.line_url, img)
                    break
            if self.left_right=="l":
                cut_img=img[0:img.shape[0],0:x1]
            elif self.left_right=="r":
                cut_img=img[0:img.shape[0],x1:img.shape[1]]
            cv2.imwrite(self.cut_url, cut_img)
        return 
    
    def normalImg(self):
        img=self.readImg()
        canny=cv2.Canny(img,120,300)
        cv2.imwrite(self.canny_url, canny)
        lines=cv2.HoughLines(canny, 1,np.pi/180,50,100,10)
        if lines is not None:
            for l in lines:
                rho,theta=l[0]
                x=np.cos(theta)
                y=np.sin(theta)
                if x==1:
                    x0=rho/x
                    y0=y*rho
                    x1=int(x0+2250*(-y))
                    y1 = int(y0 + 2250 * (x))
                    x2 = int(x0 - 2250 * (-y))
                    y2 = int(y0 - 2250 * (x))
                    cv2.line(img, (x1, y1), (x2, y2), (0 ,255,0 ), 1)
                    cv2.imwrite(self.line_url, img)
                    break
            if self.left_right=="l":
                cut_img=img[0:y1,0:x1]
            elif self.left_right=="r":
                cut_img=img[0:y1,x1:4096]
            cv2.imwrite(self.cut_url, cut_img)
        else:
            print(None)
        return 
        

CutImg 直线边缘检测、裁剪

  7个参数：

    Img_url：图片名

    Left_right：l 保留左侧，r 保留右侧

    ThetaSmall：canny 小参数，将过滤出的大边缘，在小参数值范围内检测是否连接
    
    ThetaBig：canny 大参数，过滤出大边缘

    PxNum：要检测的特征像素总量不少于

    LenNum：要检测的特征长度不少于

    GapNum：要检测的特征之间的间隔不少于

In [978]:
# C=CutImg("8y","l",0,0,0,0,0)
# C.shapeImg()

D=CutImg("7y","r",0,0,0,0,0)
D.normalImg()

# E=CutImg("4y","r",120,300,50,100,10)
# E.setImg()

In [699]:
class DetectImg:
    def __init__(self,Img_url,Small_diff,Big_diff,Filter_num,Mask,Detect_type):
        self.img_url = "D:/Data/DigImagDetct/{}.bmp".format(Img_url)
        self.dataCsv_url = "BigImageResult/{}datas.csv".format(Img_url)
        self.result_url = "BigImageResult/{}resultBW.bmp".format(Img_url)
        self.resultImg_url ="BigImageResult/{}resultBBox.bmp".format(Img_url)
        self.anchor = 0 #框体大出缺陷范围
        self.small_diff = Small_diff
        self.big_diff = Big_diff
        self.filter_num = Filter_num
        self.mask = Mask
        self.detect_type = Detect_type
    
    #读取图片函数
    def readImg(self):
        img = cv2.imread(self.img_url, 0)
        return img


    def countsNum(self):
        # 取众数
        mask_img = self.readImg().flatten()
        counts = np.bincount(mask_img)
        mask = np.argmax(counts)
        return mask

    def meanNum(self):
        #取均值
        mask_img = img.flatten()
        mask = np.mean(mask_img)
        return mask

    def maskImg(self):
        #做蒙板
        x,y = self.readImg().shape[0],self.readImg().shape[1]
        zeros_img = np.zeros(shape=(x, y))
        if self.mask=="mean":
            m=self.meanNum()
        elif self.mask=="counts":
            m=self.countsNum()
        mask_img = m + zeros_img
        return mask_img,x,y,zeros_img
    
    def findBW(self):
        if self.detect_type=="BW":
            # 进行像素级差值计算，黑白都过滤出来
            mask_img,x,y,zeros_img=self.maskImg()
            img=self.readImg()
            img_255= np.zeros(shape=(x, y))+255
            qx_array=abs(img-mask_img)
            qx_array=np.where((qx_array>self.small_diff)&(qx_array<self.big_diff),img_255,zeros_img)
            return qx_array
            
        elif self.detect_type=="B":
            # 偏黑过滤出来（油斑）
            mask_img,x,y,zeros_img=self.maskImg()
            img=self.readImg()
            img_255= np.zeros(shape=(x, y))+255
            qx_array=(img_255-img)-mask_img
            qx_array=np.where((qx_array>self.small_diff)&(qx_array<self.big_diff),img_255,zeros_img)
            return qx_array
            
        elif self.detect_type=="W":
             #偏白过滤出来
            mask_img,x,y,zeros_img=self.maskImg()
            img=self.readImg()
            img_255= np.zeros(shape=(x, y))+255
            qx_array=img-mask_img
            qx_array=np.where((qx_array>self.small_diff)&(qx_array<self.big_diff),img_255,zeros_img)
            return qx_array
        
        elif self.detect_type=="YB":
            # 油斑
            self.filter_num = 3000
            mask_img,x,y,zeros_img=self.maskImg()
            img=self.readImg()
            img_255= np.zeros(shape=(x, y))+255
            qx_array=(img_255-img)-mask_img
            qx_array=np.where((qx_array>70)&(qx_array<85),img_255,zeros_img)
            return qx_array
            
        elif self.detect_type=="GY":
            #辊印
            self.filter_num = 20
            mask_img,x,y,zeros_img=self.maskImg()
            img=self.readImg()
            img_255= np.zeros(shape=(x, y))+255
            qx_array=abs(img-mask_img)
            qx_array=np.where((qx_array>20)&(qx_array<255),img_255,zeros_img)
            return qx_array
            
        elif self.detect_type=="ZH":
            #折痕
            self.filter_num = 5
            mask_img,x,y,zeros_img=self.maskImg()
            img=self.readImg()
            img_255= np.zeros(shape=(x, y))+255
            qx_array=img-mask_img
            qx_array=np.where((qx_array>1)&(qx_array<255),img_255,zeros_img)
            return qx_array


    #保存图片函数
    def saveImg(self):
        mask_img,x,y,zeros_img=self.maskImg()
        img=self.readImg()
        #连通域：像素点总数、质心坐标、宽、高
        list_coords=[]#缺陷区域内点的坐标
        list_bbox=[]#缺陷外界边框
        labels_list = []
        areas = []#区域内像素点总数
        x = []#	质心坐标
        y = []#	质心坐标
        w = []#边界外接框(min_row, min_col, max_row, max_col)
        h = []#边界外接框(min_row, min_col, max_row, max_col)
        num=0
        qx_array=self.findBW()
        labels =measure.label(qx_array,connectivity=1)
        properties = measure.regionprops(labels)
        for prop in properties:
            if prop.area >self.filter_num:#过滤缺陷大小
                areas.append(prop.area)
                x.append(prop.centroid[0])
                y.append(prop.centroid[1])
                w.append(prop.bbox[2]-prop.bbox[0])
                h.append(prop.bbox[3]-prop.bbox[1])
                list_bbox.append(prop.bbox)#画缺陷框
                list_coords.append(prop.coords)
        out_data=pd.DataFrame(np.array([areas,x,y,w,h]),index=['像素点总数','质心坐标x','质心坐标y','缺陷x轴长','缺陷y轴长']).T
        out_data.sort_values('像素点总数',ascending=False,inplace=True)
        labels_list=[i for i in range(len(x))]
        out_data.insert(0,'序号',labels_list)
        out_data.to_csv(self.dataCsv_url,sep=',',index=None)

        # 缺陷二值化图
        for coord in list_coords:
            for c in coord:
                zeros_img[c[0]][c[1]]=255
        zeros_img.resize(zeros_img.shape[0],zeros_img.shape[1],1)
        cv2.imwrite(self.result_url, zeros_img)
        source_img=cv2.imread(self.img_url, 1)

        for i in range(len(list_bbox)):
                label_num=labels_list[i]
                bbox=list_bbox[i]
                Y = np.array([bbox[0]-self.anchor, bbox[0]-self.anchor, bbox[2]+self.anchor, bbox[2]+self.anchor])
                X = np.array([bbox[1]-self.anchor, bbox[3]+self.anchor, bbox[3]+self.anchor, bbox[1]-self.anchor])
                rr, cc = draw.polygon_perimeter(Y, X)
                draw.set_color(source_img, [rr, cc], [0, 0, 255])
        #         cv2.putText(source_img,str(label_num),(bbox[3]+20,bbox[2]+20),cv2.FONT_HERSHEY_SIMPLEX, 1,(0,0,255),2)#cv2.FONT_HERSHEY_COMPLEX
        cv2.imwrite(self.resultImg_url, source_img)
        return


DetectImg 检测缺陷、油斑、辊印、折痕

  6个参数：

    Img_url：图片名

    Small_diff：最小灰度差异值

    Big_diff：最大灰度差异值

    Filter_num：过滤Px总量大于多少Px

    Mask：mask=="mean"均值，mask=="counts"众数

    Detect_type：检测类别名，简称。

In [700]:
# Detect=DetectImg(1,0,0,0,"counts","YB")
# Detect.saveImg()

# Detect=DetectImg(2,0,0,0,"counts","YB")
# Detect.saveImg()

# Detect=DetectImg(5,0,0,0,"counts","ZH")
# Detect.saveImg()

Detect=DetectImg("9y",0,0,0,"counts","GY")
Detect.saveImg()